# TrustyAI

## Load data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv("../../datasets/FICO/heloc_dataset_v1.csv")

In [3]:
import utils

bounds = utils.data_bounds(df)

## Load model

In [4]:
from xgboost.sklearn import XGBClassifier

model = XGBClassifier()

In [5]:
model.load_model("../../models/xgboost.json")

## Counterfactuals search

### Selecting inputs

In [6]:
X = utils.get_negative_closest(model, 0.75)

In [8]:
X.to_dict()

{'ExternalRiskEstimate': 65,
 'MSinceOldestTradeOpen': 43,
 'MSinceMostRecentTradeOpen': 2,
 'AverageMInFile': 26,
 'NumSatisfactoryTrades': 15,
 'NumTrades60Ever2DerogPubRec': 1,
 'NumTrades90Ever2DerogPubRec': 1,
 'PercentTradesNeverDelq': 100,
 'MSinceMostRecentDelq': -7,
 'MaxDelq2PublicRecLast12M': 7,
 'MaxDelqEver': 8,
 'NumTotalTrades': 17,
 'NumTradesOpeninLast12M': 2,
 'PercentInstallTrades': 25,
 'MSinceMostRecentInqexcl7days': 0,
 'NumInqLast6M': 4,
 'NumInqLast6Mexcl7days': 4,
 'NetFractionRevolvingBurden': 31,
 'NetFractionInstallBurden': 83,
 'NumRevolvingTradesWBalance': 5,
 'NumInstallTradesWBalance': 1,
 'NumBank2NatlTradesWHighUtilization': 0,
 'PercentTradesWBalance': 75}

In [11]:
model.predict_proba(X.to_numpy().reshape(1, -1))

array([[0.7499573 , 0.25004265]], dtype=float32)

### Define problem

In [12]:
import trustyai
import os
import site

DEFAULT_DEP_PATH = os.path.join(site.getsitepackages()[0], "trustyai", "dep")

CORE_DEPS = [
    f"../../deps/*",
    f"{DEFAULT_DEP_PATH}/org/slf4j/slf4j-api/1.7.30/slf4j-api-1.7.30.jar",
    f"{DEFAULT_DEP_PATH}/org/apache/commons/commons-lang3/3.12.0/commons-lang3-3.12.0.jar",
    f"{DEFAULT_DEP_PATH}/org/optaplanner/optaplanner-core-impl/8.18.0.Final/"
    f"optaplanner-core-impl-8.18.0.Final.jar",
    f"{DEFAULT_DEP_PATH}/org/apache/commons/commons-math3/3.6.1/commons-math3-3.6.1.jar",
    f"{DEFAULT_DEP_PATH}/org/kie/kie-api/8.18.0.Beta/kie-api-8.18.0.Beta.jar",
    f"{DEFAULT_DEP_PATH}/io/micrometer/micrometer-core/1.8.2/micrometer-core-1.8.2.jar",
]

trustyai.init(path=CORE_DEPS)

In [17]:
from org.kie.kogito.explainability.model import PredictionInput, PredictionOutput
from trustyai.model import output
import numpy as np

TARGET = "RiskPerformance"

def predict(inputs):
    values = [_feature.value.as_number() for _feature in inputs[0].features]
    result = model.predict_proba(np.array([values]))
    bad_prob, good_prob = result[0]
    if bad_prob > good_prob:
        _prediction = (0, bad_prob)
    else:
        _prediction = (1, good_prob)
    _output = output(name=TARGET, dtype="number", value=_prediction[0], score=_prediction[1])
    return [PredictionOutput([_output])]

In [18]:
from trustyai.model import Model

provider = Model(predict)

In [19]:
from trustyai.model import feature

input_feature = []
input_dict = X.to_dict()
for name in input_dict:
    input_feature.append(
        feature(name=name, value=input_dict[name], dtype="number",
                domain=(bounds['min'][name], bounds['max'][name] + 20))
    )

In [20]:
predict([PredictionInput(input_feature)])[0].outputs.get(0).toString()

'Output{value=0, type=number, score=0.7499573230743408, name='RiskPerformance'}'

In [21]:
goal = [output(name=TARGET, dtype="number", value=1)]

In [22]:
goal[0].toString()

'Output{value=1, type=number, score=1.0, name='RiskPerformance'}'

In [23]:
from trustyai.model import counterfactual_prediction

prediction = counterfactual_prediction(
    input_features=input_feature,
    outputs=goal)

In [25]:
from trustyai.explainers import CounterfactualExplainer

explainer = CounterfactualExplainer(steps=10_000)

In [26]:
explanation = explainer.explain(prediction, provider)

In [27]:
expl_features = [e.asFeature() for e in explanation.entities]

In [28]:
predict([PredictionInput(expl_features)])[0].outputs.get(0).toString()

'Output{value=1, type=number, score=0.514225959777832, name='RiskPerformance'}'

In [29]:
def show_changes(explanation, original):
    entities = explanation.entities
    N = len(original)
    for i in range(N):
        name = original[i].name
        original_value = original[i].value.as_number()
        new_value = entities[i].as_feature().value.as_number()
        if original_value != new_value:
            print(f"Feature '{name}': {original_value} -> {new_value}")


show_changes(explanation, input_feature)

Feature 'ExternalRiskEstimate': 65.0 -> 78.0
